### El problema de los misioneros y los caníbales en AIMA

In [52]:
# importamos las cosas que vamos a usar de aima
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        # cada accion tiene un texto para identificar al operador y despues una tupla con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0,1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)

    def myHeuristic(self,node):
        return node.state[0]*3+node.state[1]*2

In [53]:
def resuelve_misioneros(problema, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """

    p8p=Problema_con_Analizados(problema)
    if p8p.check_solvability(p8p.initial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")

In [54]:
class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """

        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        
        return inversion % 2 == 0   

In [55]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado meta que esperamos
estado = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

In [56]:
%%time
# le decimos a aima que resuelva nuestro problema con el metodo de busqueda en amplitud
resuelve_misioneros(estado,breadth_first_graph_search)

Solución: [('2c', (0, 2)), ('1c', (0, 1)), ('2c', (0, 2)), ('1c', (0, 1)), ('2m', (2, 0)), ('1m1c', (1, 1)), ('2m', (2, 0)), ('1c', (0, 1)), ('2c', (0, 2)), ('1c', (0, 1)), ('2c', (0, 2))]
Algoritmo: breadth_first_graph_search
Longitud de la solución: 11. Nodos analizados: 15
Wall time: 0 ns


In [57]:
breadth_first_graph_search(estado).solution()

[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

In [58]:
%%time
# le decimos a aima que resuelva nuestro problema con el metodo de busqueda en profundidad
resuelve_misioneros(estado,depth_first_graph_search)

Solución: [('1m1c', (1, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('1c', (0, 1)), ('2m', (2, 0)), ('1m1c', (1, 1)), ('2m', (2, 0)), ('1c', (0, 1)), ('2c', (0, 2)), ('1m', (1, 0)), ('1m1c', (1, 1))]
Algoritmo: depth_first_graph_search
Longitud de la solución: 11. Nodos analizados: 12
Wall time: 0 ns


In [59]:
depth_first_graph_search(estado).solution()

[('1m1c', (1, 1)),
 ('1m', (1, 0)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1m', (1, 0)),
 ('1m1c', (1, 1))]

In [60]:
%%time
# le decimos a aima que resuelva nuestro problema con el metodo de busqueda de coste constanteS
resuelve_misioneros(estado,uniform_cost_search)

Solución: [('1m1c', (1, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('1c', (0, 1)), ('2m', (2, 0)), ('1m1c', (1, 1)), ('2m', (2, 0)), ('1c', (0, 1)), ('2c', (0, 2)), ('1c', (0, 1)), ('2c', (0, 2))]
Algoritmo: uniform_cost_search
Longitud de la solución: 11. Nodos analizados: 15
Wall time: 0 ns


In [61]:
uniform_cost_search(estado).solution()

[('1m1c', (1, 1)),
 ('1m', (1, 0)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

### Ejercicio opcional. Define alguna heurística y estudia las propiedades del algoritmo A*

In [62]:
%%time
# le decimos a aima que resuelva nuestro problema con el metodo de busqueda de coste constanteS
resuelve_misioneros(estado,astar_search,estado.myHeuristic)

Solución: [('1m1c', (1, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('1c', (0, 1)), ('2m', (2, 0)), ('1m1c', (1, 1)), ('2m', (2, 0)), ('1c', (0, 1)), ('2c', (0, 2)), ('1c', (0, 1)), ('2c', (0, 2))]
Algoritmo: astar_search
Heurística: myHeuristic
Longitud de la solución: 11. Nodos analizados: 14
Wall time: 0 ns
